In [ ]:
import sys
sys.path.appaend("/home/hyunho/sfda/")

In [1]:
from model.segmentors import EncoderDecoder
from model.configs import MODEL
from mmengine.config import ConfigDict
import torch


conf = ConfigDict(MODEL)
model = EncoderDecoder(backbone=conf.backbone, decode_head=conf.decode_head, test_cfg=conf.test_cfg)
model.load_state_dict(torch.load("/home/hyunho/sfda/deeplabv3plus_r50-d8_512x1024_40k_cityscapes_20200605_094610-d222ffcd.pth", weights_only=True)["state_dict"], strict=False)


/home/hyunho/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \
/home/hyunho/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


_IncompatibleKeys(missing_keys=[], unexpected_keys=['auxiliary_head.conv_seg.weight', 'auxiliary_head.conv_seg.bias', 'auxiliary_head.convs.0.conv.weight', 'auxiliary_head.convs.0.bn.weight', 'auxiliary_head.convs.0.bn.bias', 'auxiliary_head.convs.0.bn.running_mean', 'auxiliary_head.convs.0.bn.running_var', 'auxiliary_head.convs.0.bn.num_batches_tracked'])

In [3]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# 이미지 파일 열기
img_path = "/home/hyunho/sfda/data/gta5_dataset/images/train/00001.png"  # 이미지를 테스트할 경로
img = Image.open(img_path)

# 이미지 크기를 512x1024로 변경하고 텐서로 변환
transform = transforms.Compose([
    transforms.CenterCrop((512, 1024)),  # 크기 조정
    transforms.ToTensor()            # 텐서로 변환
])

img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)

In [5]:
model.eval()
output = model(img_tensor)

In [6]:
img.size

(1914, 1052)

In [7]:
img_tensor.shape

torch.Size([1, 3, 512, 1024])

In [8]:
model.eval()
output = model.inference(img_tensor, [{"img_shape": (512, 1024), "ori_shape": (1914, 1052)}])

In [9]:
output.shape

torch.Size([1, 19, 512, 1024])

In [ ]:
import torch
import matplotlib.pyplot as plt

# Remove the batch dimension (1, 19, 128, 256) -> (19, 128, 256)
tensor = output.squeeze(0)
classes = [
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole',
                 'traffic light', 'traffic sign', 'vegetation', 'terrain',
                 'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train',
                 'motorcycle', 'bicycle'
]
# Visualize each channel
for i in range(tensor.size(0)):  # Loop through each channel
    plt.imshow(tensor[i].cpu().detach().numpy(), cmap='gray')  # Convert tensor to numpy and visualize as grayscale
    plt.title(f'{classes[i]}')
    plt.axis('off')  # Turn off axis
    plt.savefig(f'channel_{i+1}.png')  # Save each channel as a PNG file
    plt.show()  # Show the image in case you want to view it during the process


In [4]:
from PIL import Image
import numpy as np

label_path = "/home/hyunho/sfda/data/gta5_dataset/images/train/00001.png"
label = Image.open(label_path)
print(label.size)
print(np.asarray(label).shape)
print(label.mode)

(1914, 1052)
(1052, 1914, 3)
RGB


(1914, 1052)